In [ ]:
"""
使用条件：
1. 假定图片缺口位置已经确认（缺口圆以图片中心点为圆心）
2. 缺口大小已经确认（13个像素点）
3. 图片尺寸已确定（底图的缺口和圆形图大小一致）
4. 假定图片都是是已经经过标准化处理的（同时满足上述三个条件）

"""

In [3]:
import numpy as np
import cv2

In [2]:
def read_image_grayscale(image_path):
    """
    读取图像并将其转换为灰度模式。

    Args:
        image_path (str): 图像文件的路径。

    Returns:
        numpy.ndarray: 灰度图像数据。
    """
    # 读取图像
    image = cv2.imread(image_path)

    # 如果无法读取图像,则返回None
    if image is None:
        print(f"无法读取图像: {image_path}")
        return None

    # 将图像转换为灰度模式
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return gray_image


In [75]:
def get_average_vector(c, r, a, array):
    """
    计算在距离中心坐标c为半径r的矩阵中,每隔a度的最近四个值的平均值,并将结果存储在一个向量中。

    Args:
        c (tuple): 中心坐标,形式为(x, y)。
        r (float): 半径。
        a (float): 角度,范围为0到360度。
        array (numpy.ndarray): 二维矩阵。

    Returns:
        numpy.ndarray: 存储平均值的向量。
    """
    # 将角度转换为弧度
    angle_rad = np.radians(a)

    # 计算角度增量
    angle_increment = angle_rad

    # 初始化向量
    vector = []

    # 循环计算每个角度的平均值
    while angle_increment < 2 * np.pi:
        # 计算当前角度对应的坐标
        x = c[0] + r * np.cos(angle_increment)
        y = c[1] + r * np.sin(angle_increment)

        max_x = int(np.ceil(x))
        max_y = int(np.ceil(y))

        x1 = max_x - 1
        x2 = max_x + 1
        y1 = max_y - 1
        y2 = max_y + 1

        # 最近的几个坐标点值
        sub_array = []
        for j in range(x1, x2 + 1):
            for k in range(y1, y2 + 1):
                sub_array.append(array[j, k])


        # 去掉白色
        sub_array = [x for x in sub_array if x <= 254]


        # 计算最近坐标点的平均值
        average = sum(sub_array) / len(sub_array)

        # 将平均值添加到向量中
        vector.append(average)

        # 更新角度增量
        angle_increment += angle_rad

    return np.array(vector)

In [ ]:
def find_optimal_offset(a, b):
    """
    给定相同维度的一维向量a和b,找到偏移量offset,使得偏移后的b'与a的各分量差值的方差最小。
    
    参数:
    a (numpy.ndarray): 一维向量
    b (numpy.ndarray): 一维向量,与a维度相同
    
    返回:
    offset (float): 最优偏移量
    """
    def offset_v(vec, offset):
        v = vec.tolist()
        v = v[offset:] + v[0:offset][::-1]
        return np.array(v)

    
    # 计算所有可能的偏移量
    offsets = len(a)
    
    # 计算每个偏移量对应的方差
    variances = np.array([(a - offset_v(b, offset)).var() for offset in range(offsets)])
    
    # 找到最小方差对应的偏移量
    optimal_offset = np.argmin(variances)
    
    return optimal_offset

a = np.array([1, 2, 3, 4, 5])
b = np.array([3, 4, 5, 1, 2])

offset = find_optimal_offset(a, b)
print(f"最优偏移量为: {offset}")

In [5]:
# 图片地址
# 更换链接可做测试
base_pic = "~/jd_test/base_20240522171600.jpg"

circle_pic = "~/jd_test/circle_20240522171743.jpg"



In [12]:
# 灰度模式
circle_array = read_image_grayscale(circle_pic)

base_array = read_image_grayscale(base_pic)



In [81]:
# 转为一维向量

# 半径
r = 13

# 间隔角度
a = 1

# 圆形图
circle_v = get_average_vector((80,80),r,a,circle_array)

# 底图
base_v = get_average_vector((160,100),r,a,base_array)



In [86]:
offset = find_optimal_offset(base_v, circle_v)
print(f"对应逆时针角度:{int(360 * (offset / (360 / a)))}")

对应逆时针角度:86
